In [1]:
import re
import logging
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math
import time
import pickle
from collections import defaultdict
from utils.utils import prepare
from data.configs.demo import config
from collections import defaultdict

from transformers import AutoTokenizer, AutoModel, BertModel, BertConfig
from utils.Manager import Manager

from models.Embeddings.BERT import BERT_Embedding
from models.Encoders.CNN import CNN_Encoder,CNN_User_Encoder
from models.Encoders.RNN import RNN_Encoder,RNN_User_Encoder
from models.Encoders.MHA import MHA_Encoder, MHA_User_Encoder
from models.Modules.DRM import Matching_Reducer, Slicing_Reducer
from models.Rankers.BERT import BERT_Onepass_Ranker, BERT_Original_Ranker
from models.Rankers.CNN import CNN_Ranker
from models.Encoders.Pooling import Attention_Pooling, Average_Pooling

from models.Encoders.BERT import BERT_Encoder
from models.Encoders.Pooling import *

from models.ESM import ESM
from models.TTMS import TTMS
 
from models.Modules.Attention import MultiheadAttention, get_attn_mask, XSoftmax
torch.set_printoptions(threshold=100000)

In [28]:
a = torch.rand((1,3),requires_grad=True)
b = F.softmax(a,dim=-1)
b.retain_grad()
loss = (b**2).sum()
loss.backward()
c = a.grad.clone()

softmax_back(2*b, b),c

/tmp/ipykernel_342943/2847159591.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  p = F.softmax(z)


(tensor([[-0.0452, -0.0274,  0.0726]], grad_fn=<ViewBackward>),
 tensor([[-0.0446, -0.0319,  0.0764]]))

In [ ]:
def softmax_back(da, z):
    # z, da shapes - (m, n)
    m, n = z.shape
    p = F.softmax(z, dim=-1)
    # First we create for each example feature vector, it's outer product with itself
    # ( p1^2  p1*p2  p1*p3 .... )
    # ( p2*p1 p2^2   p2*p3 .... )
    # ( ...                     )
    tensor1 = torch.einsum('ij,ik->ijk', p, p)  # (m, n, n)
    # Second we need to create an (n,n) identity of the feature vector
    # ( p1  0  0  ...  )
    # ( 0   p2 0  ...  )
    # ( ...            )
    tensor2 = torch.einsum('ij,jk->ijk', p, torch.eye(n, n))  # (m, n, n)
    # Then we need to subtract the first tensor from the second
    # ( p1 - p1^2   -p1*p2   -p1*p3  ... )
    # ( -p1*p2     p2 - p2^2   -p2*p3 ...)
    # ( ...                              )
    dSoftmax = tensor2 - tensor1
    # Finally, we multiply the dSoftmax (da/dz) by da (dL/da) to get the gradient w.r.t. Z
    dz = torch.einsum('ijk,ik->ij', dSoftmax, da)  # (m, n)
    return dz

In [17]:
a = torch.rand((1,3),requires_grad=True)
loss = (a**2).sum()
loss.backward()
a.grad, 2*a

(tensor([[0.2950, 1.8553, 1.8755]]),
 tensor([[0.2950, 1.8553, 1.8755]], grad_fn=<MulBackward0>))

In [5]:
# config.reducer = 'entity'
# config.embedding = 'deberta'
# config.bert = 'microsoft/deberta-base'
# config.device = 0

manager = Manager(config)
loaders = prepare(manager)
X1 = list(loaders[0])
X2 = list(loaders[1])
x1 = X1[0]
x2 = X2[0]

[2021-09-12 13:08:37,354] INFO (utils.utils) Hyper Parameters are 
{
    "scale": "demo",
    "mode": "tune",
    "batch_size": 5,
    "k": 5,
    "threshold": -Infinity,
    "abs_length": 40,
    "signal_length": 100,
    "his_size": 50,
    "cdd_size": 5,
    "impr_size": 10,
    "dropout_p": 0.2,
    "lr": 0.0001,
    "bert_lr": 3e-05,
    "embedding": "bert",
    "encoderN": "cnn",
    "encoderU": "rnn",
    "selector": "sfi",
    "reducer": "matching",
    "ranker": "onepass",
    "embedding_dim": 768,
    "hidden_dim": 384,
    "world_size": 0,
    "seeds": 42,
    "granularity": "avg",
    "full_attn": true,
    "ascend_history": false,
    "save_pos": false,
    "sep_his": false,
    "diversify": false,
    "no_dedup": false,
    "no_order_embed": false,
    "no_rm_punc": false,
    "no_debias": false,
    "scheduler": "linear",
    "warmup": 100,
    "shuffle": false,
    "bert": "bert-base-uncased",
    "tb": false
}
[2021-09-12 13:08:37,355] INFO (utils.utils) preparing data

In [6]:
embedding = BERT_Embedding(manager)

encoderN = CNN_Encoder(manager)
# encoderN = RNN_Encoder(manager)
# encoderN = MHA_Encoder(manager)

# encoderU = CNN_User_Encoder(manager)
encoderU = RNN_User_Encoder(manager)
# encoderU = MHA_User_Encoder(manager)
# encoderU = Attention_Pooling(manager)
# encoderU = Average_Pooling(manager)

reducer = Matching_Reducer(manager)
# reducer = Slicing_Reducer(manager)

# ranker = CNN_Ranker(manager)
# ranker = BERT_Onepass_Ranker(manager)
# ranker = BERT_Original_Ranker(manager)

# model = TTMS(manager, embedding, encoderN, encoderU, reducer).to(manager.device)
# model = ESM(manager, embedding, encoderN, encoderU, reducer, ranker).to(manager.device)

In [ ]:
class BertSelfAttention(nn.Module):
    def __init__(self, config):
        """
        one-pass bert, where other candidate news except itself are masked
        """
        super().__init__()

        self.num_attention_heads = config.num_attention_heads
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)

        self.signal_length = config.signal_length
        self.all_length = config.cdd_size * self.signal_length
        self.term_num = config.term_num
        self.full_attn = config.full_attn

        # default to term_num = his_size * k + 1
        self.register_buffer('one_pass_attn_mask_train', torch.cat([torch.eye(config.cdd_size).repeat_interleave(repeats=self.signal_length, dim=-1).repeat_interleave(repeats=self.signal_length, dim=0), torch.ones(config.cdd_size * self.signal_length, config.term_num)], dim=-1).unsqueeze(0).unsqueeze(0), persistent=False)
        self.register_buffer('one_pass_attn_mask_eval', torch.eye(config.impr_size).repeat_interleave(repeats=self.signal_length, dim=-1), persistent=False)
        self.register_buffer('ps_term_mask', torch.ones(1,self.term_num), persistent=False)

    def transpose_for_scores(self, x):
        """
        transpose the head_num dimension, to make every head operates in parallel
        """
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        """ customized bert self attention, attending to the references

        Args:
            hidden_states: normally encoded candidate news, [batch_size, signal_length, hidden_dim]
            references: normally personalized terms, [batch_size, term_num, hidden_dim]
        """
        # [CLS] + signal_length
        if self.training:
            one_pass_mask = self.one_pass_attn_mask_train
        else:
            attn_field_length = hidden_states.size(1) - self.term_num
            cdd_size = attn_field_length // self.signal_length
            one_pass_mask = torch.cat([(self.one_pass_attn_mask_eval[:cdd_size, :cdd_size * self.signal_length]).repeat_interleave(repeats=self.signal_length, dim=0), self.ps_term_mask.expand(attn_field_length, self.term_num)], dim=-1).unsqueeze(0).unsqueeze(0)

        attn_field = hidden_states[:, :-self.term_num]

        # [batch_size, head_num, *, head_dim]
        key_layer = self.transpose_for_scores(self.key(hidden_states))
        value_layer = self.transpose_for_scores(self.value(hidden_states))
        cdd_layer = self.transpose_for_scores(self.query(attn_field))
        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(cdd_layer, key_layer.transpose(-1, -2))
        # [bs, hn, cdd_length, *]
        attention_scores = (attention_scores / math.sqrt(self.attention_head_size))
        attention_mask_query = one_pass_mask * attention_mask[:, :, :-self.term_num]
        # Normalize the attention scores to probabilities.
        attention_probs = XSoftmax.apply(attention_scores, attention_mask_query, -1)
        print(attention_probs)
        
        attention_probs = self.dropout(attention_probs)

        # full attention
        if self.full_attn:
            pst_layer = self.transpose_for_scores(self.query(hidden_states[:, -self.term_num:]))
            attention_scores_pst = torch.matmul(pst_layer, pst_layer.transpose(-1, -2))
            attention_scores_pst = attention_scores_pst / math.sqrt(self.attention_head_size)
            attention_mask_pst = attention_mask[:, :, -self.term_num:, -self.term_num:]
            attention_probs_pst = XSoftmax.apply(attention_scores_pst, attention_mask_pst, -1)
            attention_probs_pst = self.dropout(attention_probs_pst)
            context_layer = torch.cat([torch.matmul(attention_probs, value_layer), torch.matmul(attention_probs_pst, value_layer[:, :, -self.term_num:])], dim=-2)

        # partial attention, where ps_terms do not interact with each other
        else:
            context_layer = torch.cat([torch.matmul(attention_probs, value_layer), value_layer[:, :, -self.term_num:]], dim=-2)

        # [batch_size, signal_length, head_num, head_dim]
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        return (context_layer,)
class BERT_Onepass_Ranker(nn.Module):
    """
    one-pass bert:
        cdd1 cdd2 ... cddn [SEP] pst1 pst2 ...
    """
    def __init__(self, config):
        # confirm the hidden dim to be 768
        assert config.embedding_dim == 768
        # confirm term_num + signal_length is less than 512
        # assert config.k * config.his_size + config.his_size + config.signal_length < 512

        super().__init__()

        self.name = 'onepass-bert'
        self.signal_length = config.signal_length
        self.term_num = config.term_num + 1
        self.embedding_dim = config.embedding_dim
        self.final_dim = self.embedding_dim

        bert_config = BertConfig()
        # primary bert
        prim_bert = BertModel(bert_config).encoder
        bert_config.signal_length = self.signal_length
        bert_config.term_num = config.term_num + 1
        bert_config.cdd_size = config.cdd_size
        bert_config.impr_size = config.impr_size
        bert_config.full_attn = config.full_attn
        for l in prim_bert.layer:
            l.attention.self = BertSelfAttention(bert_config)

        bert = BertModel.from_pretrained(
            config.bert,
            cache_dir=config.path + 'bert_cache/'
        )
        prim_bert.load_state_dict(bert.encoder.state_dict())
        self.bert = prim_bert

        self.pooler = nn.Sequential(
            nn.Linear(self.embedding_dim, self.final_dim),
            nn.Tanh()
        )
        nn.init.xavier_normal_(self.pooler[0].weight)

        # [2, embedding_dim]
        self.token_type_embedding = nn.Parameter(torch.randn(2, self.embedding_dim))
        nn.init.xavier_normal_(self.token_type_embedding)

        # [SEP] token
        if config.embedding == 'bert':
            self.sep_embedding = nn.Parameter(bert.embeddings.word_embeddings(torch.tensor([102])).clone().detach().requires_grad_(True).view(1,1,self.embedding_dim))
        elif config.embedding == 'deberta':
            self.sep_embedding = nn.Parameter(bert.embeddings.word_embeddings(torch.tensor([2])).clone().detach().requires_grad_(True).view(1,1,self.embedding_dim))
        else:
            self.sep_embedding = nn.Parameter(torch.randn(1,1,self.embedding_dim))
            nn.init.xavier_normal_(self.sep_embedding)

        self.register_buffer('sep_attn_mask', torch.ones(1, 1), persistent=False)

    def forward(self, cdd_news_embedding, ps_terms, cdd_attn_mask, ps_term_mask):
        """
        calculate interaction tensor and reduce it to a vector

        Args:
            cdd_news_embedding: word-level representation of candidate news, [batch_size, cdd_size, signal_length, embedding_dim]
            ps_terms: concatenated historical news or personalized terms, [batch_size, term_num, embedding_dim]
            cdd_attn_mask: attention mask of the candidate news, [batch_size, cdd_size, signal_length]
            ps_term_mask: attention mask of the personalized terms, [batch_size, term_num]

        Returns:
            reduced_tensor: output tensor after CNN2d, [batch_size, cdd_size, final_dim]
        """
        batch_size = cdd_news_embedding.size(0)
        cdd_size = cdd_news_embedding.size(1)

        # [bs,tn,hd]
        ps_terms += self.token_type_embedding[1]

        # [bs, cs*sl, hd]
        cdd_news_embedding = cdd_news_embedding.view(batch_size, -1, self.embedding_dim)

        bert_input = torch.cat([cdd_news_embedding, self.sep_embedding.expand(batch_size, 1, self.embedding_dim), ps_terms], dim=-2)
        bert_input[:, :cdd_news_embedding.size(1) + 1] += self.token_type_embedding[0]

        # [bs, cs*sl]
        attn_mask = cdd_attn_mask.view(batch_size, -1)
        cdd_length = attn_mask.size(-1)

        attn_mask = torch.cat([attn_mask, self.sep_attn_mask.expand(batch_size, 1), ps_term_mask], dim=-1)
        attn_mask = get_attn_mask(attn_mask)

        bert_output = self.bert(bert_input, attention_mask=attn_mask).last_hidden_state[:, 0 : cdd_size * (self.signal_length) : self.signal_length].view(batch_size, cdd_size, self.embedding_dim)
        bert_output = self.pooler(bert_output)

        return bert_output

ranker = BERT_Onepass_Ranker(manager)
model = ESM(manager, embedding, encoderN, encoderU, reducer, ranker).to(manager.device)

In [2]:
# model.eval()
# a,b = model(x2)

a,b = model(x1)

NameError: name 'model' is not defined

tensor([0.3333, 0.0000, 0.3333, 0.3333])